User Experience Analysis

In [35]:
import pandas as pd
from dbconnection import create_database_connection, read_data_into_dataframe


In [36]:
engine = create_database_connection()
df=read_data_into_dataframe(engine, "xdr_data")

In [37]:
information_columns = [
    'MSISDN/Number',
    'Avg RTT DL (ms)',
    'Avg RTT UL (ms)',
    'TCP DL Retrans. Vol (Bytes)',
    'TCP UL Retrans. Vol (Bytes)',
    'Handset Type',
    'Avg Bearer TP DL (kbps)',
    'Avg Bearer TP UL (kbps)',
]
numeric_info_columns = [
    'Avg RTT DL (ms)',
    'Avg RTT UL (ms)',
    'TCP DL Retrans. Vol (Bytes)',
    'TCP UL Retrans. Vol (Bytes)',
    'Avg Bearer TP DL (kbps)',
    'Avg Bearer TP UL (kbps)'
]


In [38]:
empty_cell=df['Handset Type'].isnull().sum()
empty_cell


572

In [39]:
df_new = df[information_columns]
df_new_clean=df_new.dropna(subset=['Handset Type'])
df_new_clean

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps)
0,3.366496e+10,42.0,5.0,NaN,NaN,Samsung Galaxy A5 Sm-A520F,23.0,44.0
1,3.368185e+10,65.0,5.0,NaN,NaN,Samsung Galaxy J5 (Sm-J530),16.0,26.0
2,3.376063e+10,NaN,NaN,NaN,NaN,Samsung Galaxy A8 (2018),6.0,9.0
3,3.375034e+10,NaN,NaN,NaN,NaN,undefined,44.0,44.0
4,3.369980e+10,NaN,NaN,NaN,NaN,Samsung Sm-G390F,6.0,9.0
...,...,...,...,...,...,...,...,...
149995,3.366865e+10,313.0,9.0,16552848.0,162614.0,undefined,63420.0,1393.0
149996,3.365069e+10,32.0,0.0,NaN,NaN,Apple iPhone 8 Plus (A1897),52.0,65.0
149997,3.366345e+10,27.0,2.0,NaN,NaN,Apple iPhone Se (A1723),23.0,54.0
149998,3.362189e+10,43.0,6.0,NaN,NaN,Apple iPhone Xs (A2097),43.0,47.0


In [40]:
#identify and replace outliers in the 'numeric_info_columns' of the 'df_new_clean' DataFrame. 

lower_10th_percentile = df_new_clean[numeric_info_columns].quantile(0.01)
upper_10th_percentile = df_new_clean[numeric_info_columns].quantile(0.99)


for col in numeric_info_columns:
    outliers = (df_new_clean[col] < lower_10th_percentile[col]) | (df_new_clean[col] > upper_10th_percentile[col])
    if col == 'Avg Bearer TP DL (kbps)':
        df_new_clean.loc[outliers, col] = df_new_clean.loc[~outliers, col].mean()
df_new_clean

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps)
0,3.366496e+10,42.0,5.0,NaN,NaN,Samsung Galaxy A5 Sm-A520F,23.0,44.0
1,3.368185e+10,65.0,5.0,NaN,NaN,Samsung Galaxy J5 (Sm-J530),16.0,26.0
2,3.376063e+10,NaN,NaN,NaN,NaN,Samsung Galaxy A8 (2018),6.0,9.0
3,3.375034e+10,NaN,NaN,NaN,NaN,undefined,44.0,44.0
4,3.369980e+10,NaN,NaN,NaN,NaN,Samsung Sm-G390F,6.0,9.0
...,...,...,...,...,...,...,...,...
149995,3.366865e+10,313.0,9.0,16552848.0,162614.0,undefined,63420.0,1393.0
149996,3.365069e+10,32.0,0.0,NaN,NaN,Apple iPhone 8 Plus (A1897),52.0,65.0
149997,3.366345e+10,27.0,2.0,NaN,NaN,Apple iPhone Se (A1723),23.0,54.0
149998,3.362189e+10,43.0,6.0,NaN,NaN,Apple iPhone Xs (A2097),43.0,47.0


In [42]:
#filling missing valued in numeric informations with mean
df_clean_NaN = df_new_clean.copy()
df_clean_NaN[numeric_info_columns] = df_clean_NaN[numeric_info_columns].apply(lambda x: x.fillna(x.mean()))
df_clean_NaN

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps)
0,3.366496e+10,42.00000,5.00000,2.082609e+07,764055.136253,Samsung Galaxy A5 Sm-A520F,23.0,44.0
1,3.368185e+10,65.00000,5.00000,2.082609e+07,764055.136253,Samsung Galaxy J5 (Sm-J530),16.0,26.0
2,3.376063e+10,108.11037,17.67528,2.082609e+07,764055.136253,Samsung Galaxy A8 (2018),6.0,9.0
3,3.375034e+10,108.11037,17.67528,2.082609e+07,764055.136253,undefined,44.0,44.0
4,3.369980e+10,108.11037,17.67528,2.082609e+07,764055.136253,Samsung Sm-G390F,6.0,9.0
...,...,...,...,...,...,...,...,...
149995,3.366865e+10,313.00000,9.00000,1.655285e+07,162614.000000,undefined,63420.0,1393.0
149996,3.365069e+10,32.00000,0.00000,2.082609e+07,764055.136253,Apple iPhone 8 Plus (A1897),52.0,65.0
149997,3.366345e+10,27.00000,2.00000,2.082609e+07,764055.136253,Apple iPhone Se (A1723),23.0,54.0
149998,3.362189e+10,43.00000,6.00000,2.082609e+07,764055.136253,Apple iPhone Xs (A2097),43.0,47.0


In [43]:
#Calculate the sum of downlink and uplink for round-trip time, TCP retransmission volume and bearer throughput

df_clean_NaN['Sum RTT'] = df_clean_NaN['Avg RTT DL (ms)'] + df_clean_NaN['Avg RTT UL (ms)']
df_clean_NaN['Sum TCP Retrans. Vol (Bytes)'] = df_clean_NaN['TCP DL Retrans. Vol (Bytes)'] + df_clean_NaN['TCP UL Retrans. Vol (Bytes)']
df_clean_NaN['Sum Bearer TP'] = df_clean_NaN['Avg Bearer TP DL (kbps)'] + df_clean_NaN['Avg Bearer TP UL (kbps)']



df_clean_NaN

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),Sum RTT,Sum TCP Retrans. Vol (Bytes),Sum Bearer TP
0,3.366496e+10,42.00000,5.00000,2.082609e+07,764055.136253,Samsung Galaxy A5 Sm-A520F,23.0,44.0,47.00000,2.159014e+07,67.0
1,3.368185e+10,65.00000,5.00000,2.082609e+07,764055.136253,Samsung Galaxy J5 (Sm-J530),16.0,26.0,70.00000,2.159014e+07,42.0
2,3.376063e+10,108.11037,17.67528,2.082609e+07,764055.136253,Samsung Galaxy A8 (2018),6.0,9.0,125.78565,2.159014e+07,15.0
3,3.375034e+10,108.11037,17.67528,2.082609e+07,764055.136253,undefined,44.0,44.0,125.78565,2.159014e+07,88.0
4,3.369980e+10,108.11037,17.67528,2.082609e+07,764055.136253,Samsung Sm-G390F,6.0,9.0,125.78565,2.159014e+07,15.0
...,...,...,...,...,...,...,...,...,...,...,...
149995,3.366865e+10,313.00000,9.00000,1.655285e+07,162614.000000,undefined,63420.0,1393.0,322.00000,1.671546e+07,64813.0
149996,3.365069e+10,32.00000,0.00000,2.082609e+07,764055.136253,Apple iPhone 8 Plus (A1897),52.0,65.0,32.00000,2.159014e+07,117.0
149997,3.366345e+10,27.00000,2.00000,2.082609e+07,764055.136253,Apple iPhone Se (A1723),23.0,54.0,29.00000,2.159014e+07,77.0
149998,3.362189e+10,43.00000,6.00000,2.082609e+07,764055.136253,Apple iPhone Xs (A2097),43.0,47.0,49.00000,2.159014e+07,90.0


In [44]:
#Adjusting columns
analysis_columns = [
    'MSISDN/Number',
    'Handset Type',
    'Sum RTT',
    'Sum TCP Retrans. Vol (Bytes)',
    'Sum Bearer TP'
]

numerical_analysis_columns  = [
    'Sum RTT',
    'Sum TCP Retrans. Vol (Bytes)',
    'Sum Bearer TP'
]
df_analysis_columns = df_clean_NaN[analysis_columns]
df_analysis_columns

,MSISDN/Number,Handset Type,Sum RTT,Sum TCP Retrans. Vol (Bytes),Sum Bearer TP
0,3.366496e+10,Samsung Galaxy A5 Sm-A520F,47.00000,2.159014e+07,67.0
1,3.368185e+10,Samsung Galaxy J5 (Sm-J530),70.00000,2.159014e+07,42.0
2,3.376063e+10,Samsung Galaxy A8 (2018),125.78565,2.159014e+07,15.0
3,3.375034e+10,undefined,125.78565,2.159014e+07,88.0
4,3.369980e+10,Samsung Sm-G390F,125.78565,2.159014e+07,15.0
...,...,...,...,...,...
149995,3.366865e+10,undefined,322.00000,1.671546e+07,64813.0
149996,3.365069e+10,Apple iPhone 8 Plus (A1897),32.00000,2.159014e+07,117.0
149997,3.366345e+10,Apple iPhone Se (A1723),29.00000,2.159014e+07,77.0
149998,3.362189e+10,Apple iPhone Xs (A2097),49.00000,2.159014e+07,90.0


In [45]:
# Group by 'MSISDN/Number' and sum the values for each group
grouped_df = df_analysis_columns.groupby('MSISDN/Number')[numerical_analysis_columns].sum().reset_index()

# Display the grouped DataFrame
grouped_df

,MSISDN/Number,Sum RTT,Sum TCP Retrans. Vol (Bytes),Sum Bearer TP
0,3.360100e+10,46.00000,2.159014e+07,76.000000
1,3.360100e+10,31.00000,2.159014e+07,99.000000
2,3.360100e+10,125.78565,2.159014e+07,97.000000
3,3.360101e+10,84.00000,7.651211e+05,248.000000
4,3.360101e+10,119.00000,3.096098e+07,56844.000000
...,...,...,...,...
106851,3.379000e+10,52.00000,2.180450e+05,10365.000000
106852,3.379000e+10,40.00000,2.159014e+07,116.000000
106853,3.197021e+12,125.78565,2.159014e+07,12223.185579
106854,3.370000e+14,125.78565,2.159014e+07,33.000000
